# Minor Flood Frequency
```{glue:figure} imgTable
:scale: 10%
:align: right
```

In this notebook we'll plot up a heatmap of minor floods through years and months. This is a common visualization for understanding the timing and duration of minor flooding; this particular visualization is borrowed from {cite:t}`mcgree_climate_2023` and {cite:t}`marra_pacific_2021`.

In [1]:
%run ../0_1_setup.ipynb
import calendar

Using custom paths from config_env.py
Data directory: /Users/jfiedler/Projects/SL_Indicators/data
Output directory: /Users/jfiedler/Projects/SL_Indicators/output


Load the data. The "SL_daily_max" file was created in the [Annual Flood Frequency](notebooks/FloodFrequency.ipynb) notebook. 

In [2]:
# load the data

# Load the data with explicit decode_timedelta parameter
SL_daily_max = xr.open_dataset(data_dir / 'SL_daily_max.nc', decode_timedelta=True)
SL_daily_max.close()

hourly_data = xr.open_dataset(data_dir / 'SL_hourly_data.nc', decode_timedelta=True)
hourly_data.close()


In [3]:
# threshold_name = SL_daily_max['threshold'].attr['long_name']
threshold = SL_daily_max['threshold']
threshold.attrs['units']


'cm'

## Make a Table

In [4]:
SL_daily_max

station_id = SL_daily_max['station_id'].values[0]
station_name = SL_daily_max['station_name'].sel(station_id=station_id).item()
flood_threshold = SL_daily_max['threshold'].sel(station_id=station_id).item()
threshold_units = SL_daily_max['threshold'].attrs['units']
threshold_name = SL_daily_max['threshold'].attrs['long_name']

# Make a dataframe with years as rows and months as columns
df = SL_daily_max['flood_day'].sel(station_id=station_id).to_dataframe()
df = df.reset_index()
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df = df.drop(columns='time')

# sum the number of flood days in each month for each year
df = df.groupby(['year', 'month']).sum()

# pivot the table
df = df.pivot_table(index='year', columns='month', values='flood_day')

# rename months to month names
df.columns = calendar.month_abbr[1:]

# add a column for the total number of flood days in each year
df['Annual'] = df.sum(axis=1)



# add a row for the total number of flood days in each month, normalized the total flood days (last row, last column)
df.loc['Monthly Total (%)'] = df.sum()
df.loc['Monthly Total (%)'] = 100*df.loc['Monthly Total (%)']/df.loc['Monthly Total (%)', 'Annual']


days_exceeding_str = 'Days Exceeding ' + str(flood_threshold) + ' ' + threshold_units + ' above MHHW'
tableTitle = station_name + days_exceeding_str + ' <br>(' + threshold_name + ')'
# Apply background gradient and add a title with larger text
styled_df = df.style.background_gradient(
    cmap='Blues', axis=None, subset=(df.index[:-1], df.columns[:-1])
).format("{:.0f}").background_gradient(
    cmap='Purples', axis=None, subset=(df.index[-1], df.columns[:-1])
).format("{:.0f}").set_caption(tableTitle).set_table_styles([
    {
        'selector': 'caption',
        'props': [('font-size', '16px'), ('margin-bottom', '10px')]
    }
])

styled_df


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
year,,,,,,,,,,,,,
1993,2,1,0,0,1,0,1,1,0,0,0,1,7
1994,0,0,0,0,0,0,0,0,0,0,0,0,0
1995,0,0,0,0,0,0,0,0,0,0,2,0,2
1996,1,1,0,0,0,0,1,0,0,0,1,7,11
1997,2,0,0,0,0,0,0,4,0,0,0,1,7
1998,2,0,0,0,0,0,0,0,0,0,0,0,2
1999,0,0,0,0,0,0,0,0,0,0,0,0,0
2000,0,0,0,0,0,0,0,0,0,0,0,0,0
2001,0,0,0,0,0,3,2,0,2,0,0,3,10


### Now we'll make it pretty

In [5]:
#make a pretty pdf of the table with great_tables
from great_tables import GT,html

dfGT = df.copy()
dfGT['year'] = df.index

# put the year column first
cols = dfGT.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfGT = dfGT[cols]
dfGT.reset_index(drop=True, inplace=True)



# Create a GreatTable object
table = (GT(dfGT)
.fmt_number(columns=calendar.month_abbr[1:13], decimals=0)
.fmt_number(columns=['Annual'], decimals=0)
.tab_header(title = days_exceeding_str, subtitle = station_name)
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = list(range(len(dfGT)-1)),
            palette=["white", "#31B1E3"])  # Adjusted blue
.data_color(domain = [0,30],
            columns=calendar.month_abbr[1:13],
            rows = [-1],
            palette=["white", "purple"])
.tab_source_note(f'Data Source: NOAA CO-OPS Hourly Water Level, {threshold_name}')
)

# save table as png
tableName = station_name + '_flood_days_intra_annual.png'
savePath = os.path.join(output_dir, tableName)

# replace any commas or spaces with underscores
# savePath = savePath.replace(' ', '_')
table.save(savePath, scale=2)

# Load Image
from IPython.display import Image
imgTable = Image(filename=savePath)

# Glue the image with a name
glue("imgTable", imgTable, display=False)

table



GT(_tbl_data=                 year        Jan       Feb      Mar     Apr       May  \
0                1993   2.000000  1.000000  0.00000  0.0000  1.000000   
1                1994   0.000000  0.000000  0.00000  0.0000  0.000000   
2                1995   0.000000  0.000000  0.00000  0.0000  0.000000   
3                1996   1.000000  1.000000  0.00000  0.0000  0.000000   
4                1997   2.000000  0.000000  0.00000  0.0000  0.000000   
5                1998   2.000000  0.000000  0.00000  0.0000  0.000000   
6                1999   0.000000  0.000000  0.00000  0.0000  0.000000   
7                2000   0.000000  0.000000  0.00000  0.0000  0.000000   
8                2001   0.000000  0.000000  0.00000  0.0000  0.000000   
9                2002   2.000000  0.000000  0.00000  0.0000  0.000000   
10               2003  10.000000  0.000000  0.00000  0.0000  0.000000   
11               2004   0.000000  0.000000  0.00000  0.0000  3.000000   
12               2005   4.000000  1.000000  1.00000  0.0000  0.000000   
13               2006   1.000000  1.000000  0.00000  0.0000  0.000000   
14               2007   1.000000  0.000000  0.00000  0.0000  0.000000   
15               2008   0.000000  0.000000  0.00000  0.0000  0.000000   
16               2009   3.000000  0.000000  0.00000  0.0000  2.000000   
17               2010   2.000000  0.000000  0.00000  0.0000  0.000000   
18               2011   2.000000  0.000000  2.00000  0.0000  0.000000   
19               2012   0.000000  0.000000  0.00000  0.0000  0.000000   
20               2013   0.000000  0.000000  0.00000  0.0000  0.000000   
21               2014   3.000000  0.000000  0.00000  0.0000  0.000000   
22               2015   1.000000  0.000000  0.00000  0.0000  0.000000   
23               2016   0.000000  0.000000  0.00000  0.0000  1.000000   
24               2017   3.000000  5.000000  0.00000  3.0000  6.000000   
25               2018   7.000000  1.000000  0.00000  0.0000  0.000000   
26               2019   0.000000  0.000000  0.00000  0.0000  1.000000   
27               2020   7.000000  3.000000  3.00000  0.0000  5.000000   
28               2021  12.000000  4.000000  0.00000  0.0000  3.000000   
29               2022   6.000000  1.000000  0.00000  0.0000  1.000000   
30               2023   0.000000  2.000000  0.00000  0.0000  0.000000   
31               2024   9.000000  3.000000  1.00000  2.0000  1.000000   
32  Monthly Total (%)  13.961606  4.013962  1.22164  0.8726  4.188482   

          Jun        Jul        Aug       Sep       Oct        Nov        Dec  \
0    0.000000   1.000000   1.000000  0.000000  0.000000   0.000000   1.000000   
1    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
2    0.000000   0.000000   0.000000  0.000000  0.000000   2.000000   0.000000   
3    0.000000   1.000000   0.000000  0.000000  0.000000   1.000000   7.000000   
4    0.000000   0.000000   4.000000  0.000000  0.000000   0.000000   1.000000   
5    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
6    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
7    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   0.000000   
8    3.000000   2.000000   0.000000  2.000000  0.000000   0.000000   3.000000   
9    0.000000   0.000000   0.000000  0.000000  0.000000   0.000000   1.000000   
10   3.000000   3.000000   4.000000  1.000000  1.000000   4.000000   3.000000   
11   5.000000   5.000000   8.000000  1.000000  0.000000   5.000000   4.000000   
12   0.000000   4.000000   0.000000  0.000000  0.000000   0.000000   3.000000   
13   0.000000   0.000000   4.000000  0.000000  0.000000   1.000000   6.000000   
14   0.000000   1.000000   0.000000  0.000000  0.000000   1.000000   4.000000   
15   0.000000   4.000000   2.000000  0.000000  0.000000   3.000000   3.000000   
16   0.000000   2.000000   3.000000  0.000000  0.000000   0.000000   0.000000   
17   0.000000   0.000000   0.000000  1.000000 

```{glue:figure} imgTable
:name: "table"
:align: center
:scale: 50%
Table of sea level records exceeding a given threshold. The data are organized into years, and months. Each 'event' is considered a 'flooding day,' where at least one hour of that day exceeds the threshold. The monthly percentage shown at the bottom of the table is relative to the total number of flooding days (for all months and years in the record). 
```

---

## Citations

```{bibliography}
:style: plain
:filter: docname in docnames
```